 # Todas las cuestiones relativas al preprocesado figuran en este Collab

#### **Recordatorio**: aunque todo este preprocesado se haya hecho en grupo, se recuerda que la redacción ha de ser personalizada.

---
---

# Length Features

Aquí se recogen algunas variables relacionadas con la longitud de la frase tanto a nivel de token como a nivel de caracter.

In [455]:
import pandas as pd
train_dataframe = pd.read_csv("https://raw.githubusercontent.com/jibt1/competition_group/main/Tareas/datasets/haha_2021_train.csv", sep=',')

In [456]:
text = train_dataframe[['text']]
text = text.iloc[0:10,:]

In [457]:
def len_token(tweet):
  tweet_tokens = tweet.replace('\n', ' ').split(' ')
  return len(tweet_tokens)
def n_characters_tweet(tweet):
  tweet = tweet.replace('\n', ' ')
  return len(tweet)

def get_len_token(text):
  return list(map(len_token, text['text'].to_list()))

def get_n_characters_tweet(text):
  return list(map(n_characters_tweet, text['text'].to_list()))

def get_n_characters_tweet(text):
  return list(map(n_characters_tweet, text['text'].to_list()))

text['len_token'] = get_len_token(text)

text['n_char'] = get_n_characters_tweet(text)

text['ratio'] = text['len_token']/text['n_char']

text

,text,len_token,n_char,ratio
0,Niveles de retraso mental: \n\n— Bajo.\n— Medi...,19,86,0.220930
1,"—Vamos Luke desenfunda tu sable, demuestra tu ...",22,140,0.157143
2,"- ¿Te ofrezco algo?, ¿Agua, café, mi corazón, ...",21,112,0.187500
3,No se porqué me hago la cabeza deooos,8,37,0.216216
4,Quisiera saber que hago durante la siesta de l...,20,106,0.188679
5,La persona que te dice que no se arrepiente de...,26,120,0.216667
6,"—Buenas don Pepe, ¿me vende un litro de leche?...",19,112,0.169643
7,Meeee aburro,2,12,0.166667
8,Macri le dijo las gordas que usar calzas está ...,10,51,0.196078
9,JAVIER CHICALITO HERNANDEZ *7,4,29,0.137931


---
---


# Detector de cansinos y corrección de repeticiones innecesarias

En twitter hay mucho "pesao" con el pulgar "cansao" y lo dejan mucho tiempo en el móooooooooovil pulsando una letraaaaa. A todos estos, los consideramos unos pesaos y lo vamos a tener en cuenta. No quieren trabajar.

¿Por qué corregiremos también a los pesaos? Porque spellchecker no puede, es fácil ver esto con un ejemplo en código. 

In [458]:
def is_cansino(tweet, tolerancia:int=2):
  # Consideramos pesados a todos aquellos
  # que repitan 3 veces una letra
  """
  Esta función identifica a los pesaos
  Args:
    tolerancia (int): tolerancia al pesao
  """
  l1 = ""
  rep = 0
  for index, letra in enumerate(tweet):
      if letra == l1:
          rep += 1
      elif rep >= tolerancia:
          return l1, rep, tolerancia, 1*True
      else:
        rep = 0
        l1 = letra
  return letra, 0, tolerancia, 1*False

def get_cansinos(corpus):
  L = list(map(is_cansino, corpus))
  return [ret[3] for ret in L]

###################################
##### Correccion repeticiones #####
###################################

# Esto es beta solo corrige la primera
# tanda de repeticiones se puede mejorar

def tweet_corrector(tweet):
    letra, rep, tolerancia, bool_ = is_cansino(tweet)
    if bool_ == 1:
        tweet = tweet.replace(rep*letra, '')
    return tweet

def corpus_corrector(corpus):
    L = list(map(tweet_corrector, corpus))
    return L

text['detected_cansinos'] = get_cansinos(text['text'].to_list())

text['text_v2'] = corpus_corrector(text['text'].to_list())

text

,text,len_token,n_char,ratio,detected_cansinos,text_v2
0,Niveles de retraso mental: \n\n— Bajo.\n— Medi...,19,86,0.220930,0,Niveles de retraso mental: \n\n— Bajo.\n— Medi...
1,"—Vamos Luke desenfunda tu sable, demuestra tu ...",22,140,0.157143,0,"—Vamos Luke desenfunda tu sable, demuestra tu ..."
2,"- ¿Te ofrezco algo?, ¿Agua, café, mi corazón, ...",21,112,0.187500,1,"- ¿Te ofrezco algo?, ¿Agua, café, mi corazón, ..."
3,No se porqué me hago la cabeza deooos,8,37,0.216216,1,No se porqué me hago la cabeza deos
4,Quisiera saber que hago durante la siesta de l...,20,106,0.188679,0,Quisiera saber que hago durante la siesta de l...
5,La persona que te dice que no se arrepiente de...,26,120,0.216667,0,La persona que te dice que no se arrepiente de...
6,"—Buenas don Pepe, ¿me vende un litro de leche?...",19,112,0.169643,0,"—Buenas don Pepe, ¿me vende un litro de leche?..."
7,Meeee aburro,2,12,0.166667,1,Me aburro
8,Macri le dijo las gordas que usar calzas está ...,10,51,0.196078,0,Macri le dijo las gordas que usar calzas está ...
9,JAVIER CHICALITO HERNANDEZ *7,4,29,0.137931,0,JAVIER CHICALITO HERNANDEZ *7


In [459]:
count = 3
string = 'holaaaa'
string.replace(count*'a', '')

'hola'

## **warning** ME he dado cuenta de esto antes de irme a dormir CUIDADOOOOOOOOOO perdemos la frecuencia de los emojisssssssssssssss

In [481]:
string = "😂😂😂😂"
string.replace("😂"*3, '')

'😂'

---
---

# SpellChecker (Corrector ortográfico)

In [460]:
# SpellChecker creo que no hace falta, pero por si acaso la dejo
# !pip install SpellChecker

In [461]:
# pyspellchecker si hace falta instalarla
# !pip install pyspellchecker
from spellchecker import SpellChecker

In [462]:
spell_check  = SpellChecker(language='es', distance=100)
spell_check.correction("chicxs")

'chicos'

---
---

# Word Emoji Context Matrix. 🤣 😆 😂 

Descripción: Extracción/guardado de Emojis y depuración del Corpus

Inconvenientes detectados con los emojis:

* 1 Resulta que los emojis hemos podido verificar que countvectorizer los ignora. Es decir si las frases fueran solo emojis el shape de la matriz word context sería (1,1), con valor empty.

* 2 Aun suponiendo que exista alguna alternativa a countvectorizer muchas personas escriben palabras juntas con emojis, e.g.: 'man👨', lo que podría generar un token más del vocabulario que innecesario. Además pueden escribir varios emojis juntos 👨, 👨👨, 👨👨👨, ... generando también un problema en su identificación.

La propuesta ha sido realizar un preproceso como el siguiente. Además, para solucionar el problema de los emoticonos, en lugar de pasárselos a countvectorizer en bruto utilizamos la decodificación a texto de la librería emoji: **emoji.demojize()**

In [463]:
# %%capture
# !pip install emoji

In [464]:
import emoji

In [465]:
def extract_emojis(tweet):
  return ''.join(c for c in tweet if c in emoji.UNICODE_EMOJI_ENGLISH)

def get_emojilist(tweet):
  emojistring = extract_emojis(tweet)
  emojilist = [emoji for emoji in emojistring]
  for emoji in emojilist:
    tweet = tweet.replace(emoji, "")
  return tweet, emojilist

def get_emoji_sentence(tweet):
  tweet, emojilist = get_emojilist(tweet)
  emoji_sentence = emoji.demojize(' '.join(emojilist))
  return tweet, emoji_sentence

def filter(text_df):
  filtered_df = pd.DataFrame(map(get_emoji_sentence, text_df['text_v2'].to_list()))
  filtered_df.rename(columns={0: 'sentences_without_emojis', 1: 'emoji_sentences'}, inplace=True)
  return filtered_df

df_filter_output = filter(text)

df_cleaned_sentences = df_filter_output[['sentences_without_emojis']]
emoji_sentences = df_filter_output['emoji_sentences']

def get_emoji_vocab(emoji_sentence):
  analyzer = CountVectorizer(binary=False, analyzer='word', # stop_words=more_stop_words,
                              ngram_range=(1, 1)).build_analyzer()
  return (emoji for emoji in analyzer(emoji_sentence))

In [466]:
from sklearn.feature_extraction.text import  CountVectorizer
cv_emoji = CountVectorizer(analyzer=get_emoji_vocab)

In [467]:
try:
  word_context_emoji = cv_emoji.fit_transform(emoji_sentences.to_list())
# Just if there is no emoji
except ValueError:
  emoji_sentences.loc[0,0] = emoji.demojize("🆘")
  word_context_emoji = cv_emoji.fit_transform(emoji_sentences.to_list())
# cv_emoji.get_feature_names()

In [480]:
cv_emoji.get_feature_names()

['sos_button']

---
---

# Obtención de Stop Words

In [468]:
# %%capture
# !pip install stop_words

In [469]:
from stop_words import get_stop_words # Se añade librería para obtener las stop_word de cualquier idioma
!wget "https://raw.githubusercontent.com/jibt1/competition_group/main/Tareas/datasets/more_stop_words.txt"

# Se obtienen las stop_words en español
import pickle
with open("more_stop_words.txt", "rb") as f:
  list_test = pickle.load(f)

stop_words = get_stop_words("spanish")
more_stop_words = stop_words + list_test
more_stop_words.sort()

--2021-05-21 23:59:36--  https://raw.githubusercontent.com/jibt1/competition_group/main/Tareas/datasets/more_stop_words.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10258 (10K) [application/octet-stream]
Saving to: ‘more_stop_words.txt.8’

more_stop_words.txt 100%[===================>]  10.02K  --.-KB/s    in 0s      

2021-05-21 23:59:36 (62.6 MB/s) - ‘more_stop_words.txt.8’ saved [10258/10258]



# ELIMINACIÓN DE IDIOMAS ALEJADOS DEL ESPAÑOL

In [470]:
# !pip install spacy_langdetect

In [471]:
# frase de prueba
# text = '¿Hola cómo سمبأسكمق estáis? 종현아생일축하해'

In [472]:
import spacy
from spacy_langdetect import LanguageDetector

# sentence level language detection
dict_ = {}

nlp = spacy.load('en')
nlp.add_pipe(LanguageDetector(), name='language_detector', last=True)

# Eliminamos palabras koreanas, vietnamitas, japonesas o árabes
L = ['ko', 'vi', 'ja', 'ar']
df_cleaned_sentences_list = df_cleaned_sentences['sentences_without_emojis'].to_list()
for index, sentence in enumerate(df_cleaned_sentences_list):
  doc = nlp(sentence)
  for subsent in doc.sents:
    if subsent._.language['language'] in L:
      df_cleaned_sentences_list[index] = sentence.replace(subsent.text,'')
df_cleaned_sentences_list

['Niveles de retraso mental: \n\n— Bajo.\n— Medio.\n— Alto.\n— Elevado.\n— Regresar con tu ex.',
 '—Vamos Luke desenfunda tu sable, demuestra tu odio, que perteneces al lado oscuro\n—Señor súbase los pantalones o llamo a seguridad.\n—JAJAJA.',
 '- ¿Te ofrezco algo?, ¿Agua, café, mi corazón, mi vida entera?\n-Agua está bien\n-Pero yo creí que.\n- ¡AGUA DIJE!',
 'No se porqué me hago la cabeza deos',
 'Quisiera saber que hago durante la siesta de la cual me levanto más cansado que cuando me acosté a dormir.',
 'La persona que te dice que no se arrepiente de nada en la vida, o no toma alcohol, o no lleva el celular cuando lo hace.',
 '—Buenas don Pepe, ¿me vende un litro de leche?\n—¿Entera?\n—No, si quiere tómese un vasito pinche viejo abusivo...',
 'Me aburro',
 'Macri le dijo las gordas que usar calzas está bien.',
 'JAVIER CHICALITO HERNANDEZ *7']

---
---

# CREACION MATRIZ WORD CONTEXT DEL TEXTO DEPURADO (SIN EMOJIS y sin idiomas poco habituales)

In [473]:
from nltk.stem.snowball import SpanishStemmer
def spanish_stemmer(sentence):
    stemmer = SpanishStemmer()
    analyzer = CountVectorizer(binary=False, analyzer='word', stop_words=more_stop_words,
                               ngram_range=(1, 1)).build_analyzer()
    return (stemmer.stem(word) for word in analyzer(sentence))
    
from sklearn.feature_extraction.text import CountVectorizer
c_vec = CountVectorizer(analyzer=spanish_stemmer, stop_words=more_stop_words, lowercase=True)
# tf_idf = TfidfTransformer(smooth_idf=True, use_idf=True)
c_vec_mat = c_vec.fit_transform(df_cleaned_sentences_list)
# tf_idf_mat = tf_idf.fit_transform(c_vec_mat)
c_vec_mat.shape

(10, 73)

In [474]:
df_c_vec = pd.DataFrame(c_vec_mat.toarray(), columns=c_vec.get_feature_names())
print(df_c_vec)

   aburr  abus  acost  agu  alcohol  alto  ...  usar  vam  vasit  vend  vid  viej
0      0     0      0    0        0     1  ...     0    0      0     0    0     0
1      0     0      0    0        0     0  ...     0    1      0     0    0     0
2      0     0      0    3        0     0  ...     0    0      0     0    1     0
3      0     0      0    0        0     0  ...     0    0      0     0    0     0
4      0     0      1    0        0     0  ...     0    0      0     0    0     0
5      0     0      0    0        1     0  ...     0    0      0     0    1     0
6      0     1      0    0        0     0  ...     0    0      1     1    0     1
7      1     0      0    0        0     0  ...     0    0      0     0    0     0
8      0     0      0    0        0     0  ...     1    0      0     0    0     0
9      0     0      0    0        0     0  ...     0    0      0     0    0     0

[10 rows x 73 columns]


---
---

# Traducción Corpus

# Descripción:

Frente a la adversidad que supone que la gran mayoría de modelos desarrollados en HuggingFace se encuentran en Inglés, se ha optado por traducir el corpus al inglés con el pipeline correspondiente. Así, podemos aprovecharlos para hacer Transfer Learning.

Puesto que la fama de los transformers les antecede, consideramos que el error de los mismos de cara a las traducciones resulta asumible.

In [475]:
# !pip install transformers
# !pip install sentencepiece

In [476]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import AutoModelForSequenceClassification, pipeline
import tqdm

In [477]:
# Autotokenizador
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-es-en")

# Elección/Descarga del modelo Preentrenado
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-es-en")

In [478]:
text

,text,len_token,n_char,ratio,detected_cansinos,text_v2
0,Niveles de retraso mental: \n\n— Bajo.\n— Medi...,19,86,0.220930,0,Niveles de retraso mental: \n\n— Bajo.\n— Medi...
1,"—Vamos Luke desenfunda tu sable, demuestra tu ...",22,140,0.157143,0,"—Vamos Luke desenfunda tu sable, demuestra tu ..."
2,"- ¿Te ofrezco algo?, ¿Agua, café, mi corazón, ...",21,112,0.187500,1,"- ¿Te ofrezco algo?, ¿Agua, café, mi corazón, ..."
3,No se porqué me hago la cabeza deooos,8,37,0.216216,1,No se porqué me hago la cabeza deos
4,Quisiera saber que hago durante la siesta de l...,20,106,0.188679,0,Quisiera saber que hago durante la siesta de l...
5,La persona que te dice que no se arrepiente de...,26,120,0.216667,0,La persona que te dice que no se arrepiente de...
6,"—Buenas don Pepe, ¿me vende un litro de leche?...",19,112,0.169643,0,"—Buenas don Pepe, ¿me vende un litro de leche?..."
7,Meeee aburro,2,12,0.166667,1,Me aburro
8,Macri le dijo las gordas que usar calzas está ...,10,51,0.196078,0,Macri le dijo las gordas que usar calzas está ...
9,JAVIER CHICALITO HERNANDEZ *7,4,29,0.137931,0,JAVIER CHICALITO HERNANDEZ *7


In [479]:
def get_corpus_translation(corpus):
  from transformers import pipeline
  pline = pipeline("translation_es_to_en", model=model, tokenizer=tokenizer)

  def get_tweet_translation(tweet):
    return pline(tweet.replace('.', '<point>'))[0]['translation_text']

  return list(map(get_tweet_translation, corpus['text_v2'].to_list()))

text['en_text'] = get_corpus_translation(text)

text = text[['text', 'text_v2', 'en_text', 'len_token', 'n_char', 'ratio', 'detected_cansinos']]
text

,text,text_v2,en_text,len_token,n_char,ratio,detected_cansinos
0,Niveles de retraso mental: \n\n— Bajo.\n— Medi...,Niveles de retraso mental: \n\n— Bajo.\n— Medi...,Mental retardation levels: — Low<point> — Medi...,19,86,0.220930,0
1,"—Vamos Luke desenfunda tu sable, demuestra tu ...","—Vamos Luke desenfunda tu sable, demuestra tu ...","""Come on, Luke, unscrew your saber, show your ...",22,140,0.157143,0
2,"- ¿Te ofrezco algo?, ¿Agua, café, mi corazón, ...","- ¿Te ofrezco algo?, ¿Agua, café, mi corazón, ...","- Do I offer you something?, Water, coffee, my...",21,112,0.187500,1
3,No se porqué me hago la cabeza deooos,No se porqué me hago la cabeza deos,I don't know why I'm doing my head.,8,37,0.216216,1
4,Quisiera saber que hago durante la siesta de l...,Quisiera saber que hago durante la siesta de l...,I'd like to know what I do during the nap from...,20,106,0.188679,0
5,La persona que te dice que no se arrepiente de...,La persona que te dice que no se arrepiente de...,The person who tells you that he doesn't regre...,26,120,0.216667,0
6,"—Buenas don Pepe, ¿me vende un litro de leche?...","—Buenas don Pepe, ¿me vende un litro de leche?...","""Good Don Pepe, will you sell me a liter of mi...",19,112,0.169643,0
7,Meeee aburro,Me aburro,I'm bored.,2,12,0.166667,1
8,Macri le dijo las gordas que usar calzas está ...,Macri le dijo las gordas que usar calzas está ...,Macri told the fat ones that wearing socks is ...,10,51,0.196078,0
9,JAVIER CHICALITO HERNANDEZ *7,JAVIER CHICALITO HERNANDEZ *7,JAVIER CHICALITO HERNANDEZ *7,4,29,0.137931,0


**I'm bored** antes se traducía como **meeee bored**, así que hemos ganado en calidad.

Generar a partir de text_v2, text_v3 donde se pase text_v2 y se haga el spell_checker

# Pendiente lo último: spell checker 